### **Model 컴포넌트 실습**

In [1]:
from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user", 
            "content": "2002년 월드컵 4강 국가 알려줘"
        }
    ]
)

ChatCompletion(id='chatcmpl-BJzyzXAjQahPheBiDN4EyuhQ1HGiH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2002년 월드컵 4강에는 독일, 브라질, 터키, 한국이 포함되었습니다.결과적으로 독일과 브라질이 결승에 진출했고, 한국은 준결승에서 독일과, 터키는 브라질과 맞붙었습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744106193, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=106, prompt_tokens=31, total_tokens=137, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [2]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name = 'gpt-4o-mini'
)
chat.invoke("안녕~ 너를 소개해줄래?")

AIMessage(content='안녕하세요! 저는 AI 언어 모델로, 다양한 주제에 대해 질문에 답하고 정보를 제공하는 역할을 하고 있어요. 글쓰기, 정보 검색, 언어 번역, 그리고 여러 가지 주제에 대해 대화할 수 있습니다. 무엇이든 궁금한 점이 있다면 언제든지 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 17, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}, id='run-b60c9203-fc87-4bcc-af0b-f73eb2650259-0', usage_metadata={'input_tokens': 17, 'output_tokens': 71, 'total_tokens': 88, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### **PromptTemplate 실습**

In [3]:
from langchain.prompts import PromptTemplate

prompt= (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고, 
        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
        )
    )

prompt 

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고, \n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [4]:
prompt.invoke({"개수":6, "재료":"사과, 잼"})

StringPromptValue(text='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 6추천하고, \n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 잼\n        ')

In [5]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
	#SystemMessage: 유용한 챗봇이라는 역할과 이름을 부여
        ("system", "You are a helpful AI bot. Your name is {name}."), 
    #HumanMessage와 AIMessage: 서로 안부를 묻고 답하는 대화 히스토리 주입
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
    #HumanMessage로 사용자가 입력한 프롬프트를 전달
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


### **LCEL로 Chain 구축하기**

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

#LLM 호출
model = ChatOpenAI(model="gpt-4o-mini")

#LCEL로 프롬프트템플릿-LLM-출력 파서 연결하기
chain = prompt | model | StrOutputParser()

#invoke함수로 chain 실행하기
chain.invoke({"topic": "ice cream"})

'What did the ice cream cone say to the refrigerator?  \n\n"Stop staring at me, you’re making me melt!"'

In [7]:
#Chain 선언
model = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

#Chain의 stream()함수를 통해 스트리밍 기능 추가
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why do polar bears never get lost?

Because they always follow the "bear"ings!

### **OutputParser 실습**

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature = 0)

#ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
model = ChatOpenAI(model="gpt-4o-mini")
chain = chat_template | model
chain.invoke("액션")

AIMessage(content='["다크 나이트", "존 윅", "매드 맥스: 분노의 도로"]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 72, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}, id='run-fccbbe9b-cabe-4033-a15a-92030d64e19f-0', usage_metadata={'input_tokens': 72, 'output_tokens': 25, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

#CSV 파서 선언
output_parser = CommaSeparatedListOutputParser()

#CSV 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [10]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

#CSV 파서 선언
output_parser = CommaSeparatedListOutputParser()

#CSV 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()

#프롬프트 템플릿의 partial_variables에 CSV 형식 지정 프롬프트 주입
prompt = PromptTemplate(
    template="List {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(model="gpt-4o-mini")

#프롬프트템플릿-모델-Output Parser를 체인으로 연결
chain = prompt | model | output_parser
chain.invoke({"subject": "공포 영화"})

['기생충',
 '곡성',
 '저주',
 '악마를 보았다',
 '사탄의 인형',
 '더 헌트',
 '미드소마',
 '유전',
 '13일의 금요일',
 '지옥',
 '더 링',
 '일말의 희망',
 '괴물',
 '숨바꼭질',
 '공포의 집']

In [11]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# 데이터 구조를 정의합니다.
class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")
    

# JsonOutputParser를 설정하고 프롬프트 템플릿에 format_instructions를 삽입합니다.
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

country_query = "아르헨티나는 어떤 나라야?"
chain.invoke({"query": country_query})

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


{'continent': 'South America', 'population': '45195777'}

### **LCEL의 Runnable 객체에 대해 알아보기**

#### **RunnablePassthrough 알아보기**

In [12]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")

'안녕하세요'

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""다음 한글 문장을 프랑스어로 번역해줘 {sentence}
French sentence: (print from here)""")

model = ChatOpenAI(model="gpt-4o-mini")

output_parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | output_parser
runnable_chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

'Elle lit un livre chaque matin.'

In [14]:
(RunnablePassthrough.assign(mult=lambda x: x["num"]*3)).invoke({"num":3})

{'num': 3, 'mult': 9}

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

#### **RunnableLambda 알아보기**

In [16]:
def add_smile(x):
    return x + ":)"

In [17]:
from langchain_core.runnables import RunnableLambda

add_smile = RunnableLambda(add_smile)

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt_str = "{topic}의 역사에 대해 세문장으로 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model_name = 'gpt-4o-mini')

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [19]:
from langchain_core.runnables import RunnableLambda

def add_thank(x):
    return x + " 들어주셔서 감사합니다 :)"

add_thank = RunnableLambda(add_thank)

In [20]:
chain = prompt | model | output_parser | add_thank
chain.invoke("반도체")

'반도체는 20세기 중반에 트랜지스터의 발명으로 시작되었으며, 이는 전자기기의 소형화와 효율성을 크게 향상시켰습니다. 이후 집적회로(IC)의 개발로 반도체 기술은 급속히 발전하였고, 컴퓨터, 통신 및 다양한 전자기기에 필수적인 요소가 되었습니다. 현재 반도체 산업은 인공지능, IoT, 자율주행차 등 혁신적인 기술의 기반을 제공하며 글로벌 경제의 중요한 축으로 자리잡고 있습니다. 들어주셔서 감사합니다 :)'

#### **RunnableParallel 알아보기**

In [21]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [22]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thank,
)

runnable.invoke("안녕하세요")

{'passed': '안녕하세요', 'modified': '안녕하세요 들어주셔서 감사합니다 :)'}

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(model = 'gpt-4o-mini', max_tokens = 128, temperature = 0)

history_prompt = ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요.")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 분야의 유명인사 3명의 이름만 알려주세요.")

output_parser = StrOutputParser()

history_chain = history_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(history=history_chain, celeb=celeb_chain)

result = map_chain.invoke({"topic": "AI"})
result

{'history': 'AI는 "Artificial Intelligence"의 약자로, 한국어로는 "인공지능"이라고 합니다. 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 등의 작업을 수행할 수 있도록 하는 기술을 의미합니다.',
 'celeb': '1. 앤드류 응 (Andrew Ng)\n2. 제프리 힌튼 (Geoffrey Hinton)\n3. 얀 르쿤 (Yann LeCun)'}